In [1]:
from abc import ABCMeta,abstractmethod
import Ipynb_importer
from WindPy import w
import math
from math import exp, sqrt
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
class StockVolatility(object):
    __metaclass__ = ABCMeta #抽象类
    @abstractmethod  #抽象方法
    def getVolatility(self):
        pass

In [3]:
class HistoricalVolatility(StockVolatility):
    
    def __init__(self, stockCode, tradingDaysPerYear, startDate, endDate):    
        self.stockCode = stockCode
        self.tradingDaysPerYear = tradingDaysPerYear
        self.startDate = startDate
        self.endDate = endDate

    def getVolatility(self):
        """
        计算标的在一段时间内的年化波动率
        stockCode: 标的代码
        startDate: 开始日期
        endDate: 结束日期
        tradingDaysPerYear: 一年交易日天数
        """
        closingPrice = w.wsd(self.stockCode, "close", self.startDate, self.endDate, "").Data[0]

        dailyReturn = [0]
        continuousCompoundReturn = [0]
        for i in range(1, len(closingPrice)):
            dailyReturn.append(closingPrice[i] / closingPrice[i - 1])
            continuousCompoundReturn.append(math.log(dailyReturn[i]))

        continuousCompoundReturnAvg = sum(continuousCompoundReturn) / len(closingPrice) # miu

        # calculate (LN(R) - miu) ^ 2
        thetaFangList = [0]
        for j in continuousCompoundReturn:
            thetaFangList.append(pow(j - continuousCompoundReturnAvg, 2))
        variance =  sum(thetaFangList) / (len(closingPrice) - 1) # 样本方差theta^2
        standardDeviation = pow(variance, 0.5)   #标准差，即波动率
        annualVolatility = standardDeviation * sqrt(self.tradingDaysPerYear)
        return annualVolatility

In [4]:
"""Test"""
OPTION_START_DATE = "2020-06-23"
OPTION_STRIKE_DATE = "2021-03-15"
STOCK_CODE = "000905.SH"
TRADING_DATES_PER_YEAR = 240
testObj = HistoricalVolatility(STOCK_CODE, TRADING_DATES_PER_YEAR, OPTION_START_DATE, OPTION_STRIKE_DATE)
testObj.getVolatility()

0.21521726624603071